## Imports

In [1]:
# pip install selenium webdriver-manager

# o objetivo deste notebook é olhar para os links que foram scrape e tentar extrair as peças do procedimento

# agora vou primeiro passar isto tudo para um repositório no git lol

import pandas as pd
import polars as pl
import numpy as np

import os

from copy import deepcopy

from utilities.schema import contracts_schema, lots_schema, bid_schema, procedimentos_schema
from utilities.funcoes import extract_sliding_windows, reverse_order_unique_contract_years, mean_per_year_per_cpv, get_covid_legal_frameworks

c:\Users\carol\Documents\bolsa\Text_Fields\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
procedimentos = pl.read_csv("../../data/impic_data/procedimentos.csv",
                            separator=";",
                            schema_overrides=procedimentos_schema(),
                            columns=[
                                "ContractingProcedureAliasID", "Número do Anúncio (Nº/Ano)", "Descrição/designação do procedimento", "Data de Criação", 
                                "URL Peças do Procedimento", "Nº de Lotes", "Regime de Contratação"
                                     ],
                            null_values=["NULL"]) \
                            .with_columns(
                                pl.col("Data de Criação").replace("NULL", None).str.split(" ").list.first().str.replace("'", "").str.to_date("%F")
                                            ) \
                            .unique(subset="ContractingProcedureAliasID")

In [3]:
contracts = pl.read_csv("../../data/impic_data/contratos.csv", separator=";", schema_overrides=contracts_schema(),
    columns=['N.º Procedimento (ID BASE)', 'N.º Contrato', 'Data da decisão adjudicação', 
             'Data Celebração', "Contratação Excluída", "Tipo de procedimento", "Medidas Especiais",
              "Ao abrigo dos critérios materiais", 'Preço BASE (€)', 'Preço Contratual (€)'], 
    null_values=["NULL"]) \
    .with_columns(
        pl.col("Data da decisão adjudicação").replace("NULL", None).str.split(" ").list.first().str.replace("'", "").str.to_date("%F"),
        pl.col('Data Celebração').replace("NULL", None).str.split(" ").list.first().str.replace("'", "").str.to_date("%F"),
    ) \
    .join(procedimentos.select(["ContractingProcedureAliasID", "Regime de Contratação"]), how="left", left_on="N.º Procedimento (ID BASE)", right_on="ContractingProcedureAliasID", coalesce=True) \
    .filter(pl.col("Contratação Excluída") == False) \
    .filter(pl.col("Tipo de procedimento").is_in(["Concurso público", "Concurso limitado por prévia qualificação"])) \
    .filter(pl.col("Medidas Especiais").is_null()) \
    .filter(~pl.col("Regime de Contratação").str.contains_any(get_covid_legal_frameworks())) \
    .unique(subset=['N.º Procedimento (ID BASE)', 'N.º Contrato'])


lots = pl.read_csv("../../data/impic_data/lotes.csv", separator=";", schema_overrides=lots_schema(),
    columns=[
       "IDAliasProc", "ContractID",
       "Preço Contratual", "Número de Ordem do Lote",
       "Valor do Lote"
    ], 
    null_values=["NULL"])


bid_level_data = pl.read_csv(
        "../../data/impic_data/anexo_vii.csv",
        separator=";",
        schema_overrides=bid_schema(),
        columns=["Número de ordem do lote em causa", "NIF Contestant", "Dados de base do procedimento"]
    ) 

# to have cpvs
procurement_contracts = pl.read_ipc("../../data/impic_data/procurement_contracts_2018_2023.arrow")

In [4]:
contratos_cleaned = pl.read_ipc("../../data/clean_up_data/contracts_cleanedup.arrow")

lotes_cleaned = pl.read_ipc("../../data/clean_up_data/lots_cleanedup.arrow")

bids_cleaned = pl.read_ipc("../../data/clean_up_data/bids_cleanedup.arrow")

In [5]:
# preprocess contracts
contracts = contracts.join(contratos_cleaned,
                                  how="anti", 
                                  on = ["N.º Procedimento (ID BASE)", "N.º Contrato"], 
                                  coalesce=True)


# preprocess lots
lots = lots.join(lotes_cleaned,
                                  how="anti", 
                                  on = ["IDAliasProc", "ContractID", "Número de Ordem do Lote"], 
                                  coalesce=True)

# preprocess bids
bids = bid_level_data.join(bids_cleaned,
                                  how = "anti",
                                  on = ["Dados de base do procedimento", "Número de ordem do lote em causa"],
                                  coalesce=True)

In [6]:
# join contracts with lots

lots_per_contract = contracts \
    .join(
        lots, 
        left_on=["N.º Procedimento (ID BASE)", "N.º Contrato"], 
        right_on=["IDAliasProc", "ContractID"], 
        how="left", 
        coalesce=True
    ) \
    .with_columns(
        [
            pl.col("Número de Ordem do Lote").fill_null(pl.lit("NL")).alias("Número de Ordem do Lote") # Fill nulls with NL ("No Lots")
        ]
    )

In [7]:
# objetivo -> não calcular preço estimado com base em informações FUTURAS

contracts = contracts.join(
        procurement_contracts.with_columns(pl.col("contract_id").cast(pl.Utf8)).select(["contract_id", "cpvs"]),
        how="inner",
        left_on=["N.º Contrato"],
        right_on=["contract_id"],
        coalesce=False
    ) \
        .with_columns(
            pl.col("cpvs").str.slice(1, length=3),
            pl.col("Data Celebração").dt.year().alias("contract_year").cast(pl.Int64)
        )

In [ ]:
def ratio_contractual_estimated_base(contracts_table):
    estimated_prices = pl.concat(
        [mean_per_year_per_cpv(contracts_table, window) for window in extract_sliding_windows(reverse_order_unique_contract_years(contracts_table, "contract_year"), 3)]
        )
    
    return contracts_table.join(estimated_prices, how="left", left_on = ["contract_year", "cpvs"], right_on=["end_period", "cpvs"], coalesce=True) \
    .with_columns(
        (pl.col("Preço Contratual (€)")/pl.col("Preço Estimado")).alias("estimated_ratio"), # preço contratual pode ser inferior ou superior ao preço estimado -> se for superior é pior 
        (pl.col("Preço Contratual (€)")/pl.col("Preço BASE (€)")).alias("base_ratio") # preço contratual é sempre igual ou inferior ao preço base -> fração vai ser sempre 1 ou menos. muito próximo de 1 é mau
    )

In [9]:
def calc_number_bidders(lots_contracts, bids):
    return lots_contracts.join(bids, left_on=["N.º Procedimento (ID BASE)", "Número de Ordem do Lote"],
                                right_on=["Dados de base do procedimento", "Número de ordem do lote em causa"]) \
                                .group_by(
                                    ["N.º Procedimento (ID BASE)", "N.º Contrato", "Número de Ordem do Lote"]
                                ) \
    .agg(pl.n_unique("NIF Contestant").alias("num_bidders"), pl.col("Tipo de procedimento").first()) \
    .unique(subset=["N.º Procedimento (ID BASE)", "N.º Contrato", "Número de Ordem do Lote"])
    #.select(["N.º Procedimento (ID BASE)", "N.º Contrato", "Número de Ordem do Lote", "num_bidders", "Tipo de procedimento"])

In [11]:
# ratio between contractual and estimated price / contractual and base price -> indicator per contract, not per lot

ratio = ratio_contractual_estimated_base(contracts)

# number of bidders, indicator per lot

number_bidders = calc_number_bidders(lots_per_contract, bids)

In [14]:
# save indicators

ratio.write_csv("../../data/ratio_val_indicator.csv")
number_bidders.write_csv("../../data/number_bidders_val_indicator.csv")

Excluir procedimentos não concursais e outros concursais mas muito raros:

- Procedimento de negociação                           35
- Contratação excluída II                              16
- Concurso de conceção simplificado                     2
- Parceria para a inovação                              2
- Concurso de ideias simplificado                       1
- Serviços sociais e outros serviços específicos        1